# Création des jeux de données

Les inputs de ce notebook sont :
- le fichier .csv de départ de l'étude
- le dossier d'images initial

les outputs de ce notebook sont :
- un dataframe (cleaned_dataset.csv) sauvegardé dans le dossier cible de l'étude contenant les données nettoyées (champignons comestibles et non comestibles)
- un dataframe (cleaned_dataset_edible.csv) sauvegardé dans le dossier cible de l'étude contenant les données nettoyées (champignons comestibles uniquement)
- un dossier d'images correspondant aux images contenues dans cleaned_dataset.csv dans un sous-dossier 'dataset_cleaned'
- un dossier d'images correspondant aux images contenues dans cleaned_dataset_edible.csv dans un sous-dossier 'edible_dataset_cleaned'


In [ ]:
from IPython.display import display
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px


### Définition du répertoire source et du dataset source

In [ ]:
target_dir = r'../../raw/'
file_to_open = 'observations_mushroom.csv'
images_dir = "../../raw/"
dataset_dir = "../../dataset/"

# Préparation du dataset d'images inital

#### Import et nettoyage du jeu de données initial 

In [ ]:
# import du dataset d'origine du use_case
df = pd.read_csv(os.path.join(target_dir, file_to_open))

# suppression des colonnes sans interet
df = df.drop(columns = ['image_id', 'observation', 'label', 'image_url', 'user',
       'date', 'gbif_info/speciesKey',
       'gbif_info/orderKey',
       'gbif_info/confidence', 'gbif_info/classKey',
       'gbif_info/matchType', 'gbif_info/familyKey', 'gbif_info/status',
       'gbif_info/usageKey', 'gbif_info/kingdomKey', 'gbif_info/genusKey',
       'gbif_info/canonicalName', 'gbif_info/phylumKey', 'gbif_info/synonym', 'gbif_info/scientificName',
       'thumbnail', 'location', 'gbif_info/note', 'gbif_info'])

# rename des colonnes de df avec des noms plus explicites
df = df.rename(columns={"gbif_info/species" : 'species',
                   "gbif_info/class" : 'classes',
                  "gbif_info/order" : 'order',
                  "gbif_info/family" : 'family',
                  "gbif_info/genus" : 'genus',
                  "gbif_info/kingdom" : 'kingdom',
                  "gbif_info/rank" : 'rank',
                  "gbif_info/phylum" : 'phylum',
                       "image_lien" : "filename"})

# reorganisation des colonnes de df
df = df.reindex(columns=['filename','kingdom', 'phylum', 'classes', 'order', 'family', 'genus', 'species'])

# suppression des lignes des df faisant référence à nue meme image
df = df.drop_duplicates(subset="filename")

# suppression des NaN de df
df.dropna(inplace = True)

# affichage de quelques informations sur le jeu de données initial nettoyé
display(df.head(), df.info())


# Creation de la colonne cible

la liste des champignons comestibles utilisées provient de la source :
ANSES : Avis de l’agence nationale de sécurité sanitaire de l’alimentation, de l’environnement et du travail relatif à « une demande d’avis lié à un projet d’arrêté relatif aux variétés comestibles de champignons de culture et sauvages » (Saisine n° 2015-SA-0180)
Disponible sur https://www.anses.fr/fr/search?search_api_views_fulltext=liste%20des%20champignons%20comestibles
Consultée pour la dernière fois le 30/11/2023

In [ ]:
# creation de la liste des champignons comestibles

list_edible = ['Agaricus arvensis',
'Agaricus bitorquis',
'Agaricus campestris',
'Agaricus essettei',
'Agaricus macrocarpus',
'Agaricus osecanus',
'Agaricus silvaticus',
'Agaricus silvicola',
'Agaricus urinascens',
'Coprinus comatus',
'Boletus mamorensis',
'Butyriboletus appendiculatus',
'Butyriboletus pseudoregius',
'Butyriboletus regius',
'Imleria badia',
'Leccinum albostipitatum',
'Leccinum aurantiacum',
'Leccinum duriusculum',
'Leccinum pseudoscabrum',
'Leccinum scabrum',
'Leccinum variicolor',
'Leccinum versipelle',
'Neoboletus luridiformis',
'Suillellus luridus',
'Suillellus queletii',
'Boletus aereus',
'Boletus aestivalis',
'Boletus edulis',
'Boletus pinophilus',
'Agaricus bisporus',
'Cantharellus alborufescens',
'Cantharellus amethysteus',
'Cantharellus cibarius',
'Cantharellus cinereus',
'Cantharellus ferruginascens',
'Cantharellus friesii',
'Cantharellus ianthinoxanthus',
'Cantharellus lutescens',
'Cantharellus melanoxeros',
'Cantharellus pallens',
'Cantharellus tubaeformis',
'Pseudocraterellus undulatus',
'Cuphophyllus pratensis',
'Cuphophyllus virgineus',
'Hygrocybe punicea',
'Hygrophorus latitabundus',
'Hygrophorus marzuolus',
'Hygrophorus nemoreus',
'Hygrophorus penarioides',
'Hygrophorus russula',
'Laccaria amethystina',
'Laccaria bicolor',
'Laccaria laccata',
'Laccaria proxima',
'Lactarius deliciosus',
'Lactarius picinus',
'Lactarius sanguifluus',
'Lactarius semisanguifluus',
'Lactarius vinosus',
'Lactarius volemus',
'Chlorophyllum rhacodes',
'Leucoagaricus leucothites',
'Macrolepiota excoriata',
'Macrolepiota mastoidea',
'Macrolepiota procera',
'Mitrophora semilibera',
'Morchella elata',
'Morchella esculenta',
'Hydnum repandum',
'Hydnum rufescens',
'Tuber brumale',
'Tuber magnatum',
'Tuber melanosporum',
'Pleurotus citrinopileatus',
'Pleurotus cornucopiae',
'Pleurotus djamor',
'Pleurotus eryngii',
'Pleurotus ostreatus',
'Pleurotus pulmonarius',
'Russula aurea',
'Russula cyanoxantha',
'Russula integra',
'Russula mustelina',
'Russula romellii',
'Russula vesca',
'Russula virescens',
'Russula xerampelina',
'Clitocybe odora',
'Infundibulicybe geotropa',
'Infundibulicybe gibba',
'Lepista flaccida',
'Lepista flaccida',
'Lepista glaucocana',
'Lepista irina',
'Lepista nuda',
'Lepista panaeolus',
'Lepista personata',
'Lepista sordida',
'Tricholoma columbetta',
'Tricholoma portentosum',
'Tricholoma atrosquamosum',
'Tricholoma atrosquamosum var. squarrulosum',
'Tricholoma cingulatum',
'Tricholoma orirubens',
'Tricholoma scalpturatum',
'Tricholoma terreum',
'Craterellus cornucopioides',
'Ptychoverpa bohemica',
'Verpa conica',
'Aleuria aurantia',
'Amanita caesarea',
'Amanita fulva',
'Amanita rubescens',
'Amanita vaginata',
'Auricularia auricula-judae',
'Auricularia polytricha',
'Calocybe gambosa',
'Calvatia gigantea',
'Clitopilus prunulus',
'Cortinarius caperatus',
'Cortinarius praestans',
'Cyclocybe aegerita',
'Disciotis venosa',
'Entoloma aprile',
'Entoloma clypeatum',
'Fistulina hepatica',
'Flammulina velutipes',
'Gomphidius glutinosus',
'Gomphus clavatus',
'Grifola frondosa',
'Guepinia helvelloides',
'Gyroporus cyanescens',
'Hygrophoropsis aurantiaca',
'Hypsizygus tessulatus',
'Kuehneromyces mutabilis',
'Laetiporus sulphureus',
'Lentinula edodes',
'Limacella guttata',
'Lyophyllum decastes',
'Marasmius oreades',
'Otidea onotica',
'Polyporus umbellatus',
'Pseudohydnum gelatinosum',
'Ramaria botrytis',
'Sparassis crispa',
'Stropharia rugosoannulata',
'Suillus luteus',
'Volvariella volvacea',
'Volvopluteus gloiocephalus']

# assignation de la valeur 1 si l'espece est comestible, 0 le cas échéant
df = df.assign(edible=df["species"].isin(list_edible))
df['edible'] = df['edible'].apply(int)

# affichage de quelques informations sur le dataset final de l'étude
display(df.head(), df.info())


#### affichage d'une figure représentant la repartition de la cible de notre étude dans le jeu de données final

In [ ]:
# attention, cette cellule est utilise de nombreuses ressources pour affichage !

#dataset = df

#dataset['edible'].replace(1, 'edible', inplace=True)
#dataset['edible'].replace(0, 'inedible', inplace=True)


#fig = px.sunburst(dataset.assign(hole=' '),
#                  path=['hole','edible', 'kingdom','phylum','classes', 'order', 'family', 'genus', 'species' ],
#                 width=750, height=750)

#fig.show()

# Nettoyage de la base de données d'images correspondant au dataframe

### Pour la classification sur la comestibilité

In [ ]:
# création d'un nouveau dossier qui contiendra les images pour la classification binaire (comestible / non-comestible)
try:
    dataset_cleaned = dataset_dir + '/dataset_cleaned'
    os.mkdir(dataset_cleaned)
except FileExistsError:
    pass


# extraction des noms d'images du dataframe
images_names = df['filename'].tolist()

missing_picture = []
# itération sur les noms d'images
for image_name in images_names :    
    # trouver l'image dans le dossier d'origine
    img_id = int(image_name[:-4])
    subdir = '0' if (img_id < 100000) else str(img_id)[0]
    original_image_path = os.path.join(images_dir, subdir, image_name)

    # copie de l'image dans le dossier de destination
    new_image_path = os.path.join(dataset_cleaned, image_name)
    if os.path.exists(original_image_path):
        shutil.copy(original_image_path, new_image_path)
    else:
        position_backslash = original_image_path.rfind("\\")
        picture = original_image_path[position_backslash + 1 :]
        missing_picture.append(picture)




In [ ]:
# controle de la correspondance entre le dataset et le dataframe
print(len(df), len(os.listdir(dataset_cleaned)))

In [ ]:
#si nécessaire, afin de nettoyer le dataframe....
'''
# suppression des lignes de df pour lesquelles il n'y a pas d images    
df = df.drop(df[df["filename"].isin(missing_picture)].index)

print(len(df), len(os.listdir(dataset_cleaned)))
'''


### Pour la classification sur la taxonomie 

In [ ]:
# création d'un nouveau dossier qui contiendra les images de champignons comestibles
os.mkdir(target_dir + '\edible_dataset_cleaned')
edible_dataset_cleaned = target_dir + '\edible_dataset_cleaned'

# extraction des noms d'images du dataframe
df_edible = df[df['edible'] == 1]
images_names_edibles = df_edible['filename'].tolist()

missing_picture = []
# itération sur les noms d'images
for image_name in images_names_edibles :    
    # trouver l'image dans le dossier d'origine
    original_image_path = os.path.join(images_dir, image_name)
    # copie de l'image dans le dossier de destination
    new_image_path = os.path.join(edible_dataset_cleaned, image_name)
    if os.path.exists(original_image_path):
        shutil.copy(original_image_path, new_image_path)
    else:
        position_backslash = original_image_path.rfind("\\")
        picture = original_image_path[position_backslash + 1 :]
        missing_picture.append(picture)

# controle de la correspondance entre le dataset et le dataframe
print(len(df_edible), len(os.listdir(edible_dataset_cleaned)))

In [ ]:
#si nécessaire, afin de nettoyer le dataframe....
'''
# suppression des lignes de df_edible pour lesquelles il n'y a pas d images    
df_edible = df_edible.drop(df_edible[df_edible["filename"].isin(missing_picture)].index)

print(len(df_edible), len(os.listdir(edible_dataset_cleaned)))
'''

## Extraction des nouveaux dataframes nettoyés

In [ ]:
## Export des dataframes nettoyés

# nom des dataframes exportés
name = "cleaned_dataset.csv"
name_edible = "cleaned_dataset_edible.csv"

# export du dataframe
df.to_csv(os.path.join(dataset_dir, name), index = False)
#df_edible.to_csv(os.path.join(target_dir, name_edible), index = False)
